In [5]:
import atlite
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
from cartopy.crs import PlateCarree as plate
import cartopy.io.shapereader as shpreader
from matplotlib.gridspec import GridSpec
import seaborn as sns
import geopandas as gpd
import yaml

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [24]:
cutout = atlite.Cutout(path="western-europe-2011-01.nc",
                       module="era5",
                       x=slice(-13.6913, 1.7712),
                       y=slice(49.9096, 60.8479),
                       time="2011-01",
                       #features = ["wave_height"]
                       )
cutout.data

INFO:atlite.cutout:Building new cutout western-europe-2011-01.nc


<xarray.Dataset>
Dimensions:  (x: 62, y: 44, time: 744)
Coordinates:
  * x        (x) float64 -13.5 -13.25 -13.0 -12.75 -12.5 ... 1.0 1.25 1.5 1.75
  * y        (y) float64 50.0 50.25 50.5 50.75 51.0 ... 60.0 60.25 60.5 60.75
  * time     (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
    lon      (x) float64 -13.5 -13.25 -13.0 -12.75 -12.5 ... 1.0 1.25 1.5 1.75
    lat      (y) float64 50.0 50.25 50.5 50.75 51.0 ... 60.0 60.25 60.5 60.75
Data variables:
    *empty*
Attributes:
    module:             era5
    prepared_features:  []
    chunksize_time:     100

In [4]:
def convert_and_aggregate(
    cutout,
    convert_func,
    matrix=None,
    index=None,
    layout=None,
    shapes=None,
    shapes_crs=4326,
    per_unit=False,
    return_capacity=False,
    capacity_factor=False,
    show_progress=True,
    dask_kwargs={},
    **convert_kwds,
):
    """
    Convert and aggregate a weather-based renewable generation time-series.

    NOTE: Not meant to be used by the user him or herself. Rather it is a
    gateway function that is called by all the individual time-series
    generation functions like pv and wind. Thus, all its parameters are also
    available from these.

    Parameters
    -----------
    matrix : N x S - xr.DataArray or sp.sparse.csr_matrix or None
        If given, it is used to aggregate the grid cells to buses.
        N is the number of buses, S the number of spatial coordinates, in the
        order of `cutout.grid`.
    index : pd.Index
        Index of Buses.
    layout : X x Y - xr.DataArray
        The capacity to be build in each of the `grid_cells`.
    shapes : list or pd.Series of shapely.geometry.Polygon
        If given, matrix is constructed as indicatormatrix of the polygons, its
        index determines the bus index on the time-series.
    shapes_crs : pyproj.CRS or compatible
        If different to the map crs of the cutout, the shapes are
        transformed to match cutout.crs (defaults to EPSG:4326).
    per_unit : boolean
        Returns the time-series in per-unit units, instead of in MW (defaults
        to False).
    return_capacity : boolean
        Additionally returns the installed capacity at each bus corresponding
        to `layout` (defaults to False).
    capacity_factor : boolean
        If True, the static capacity factor of the chosen resource for each
        grid cell is computed.
    show_progress : boolean, default True
        Whether to show a progress bar.
    dask_kwargs : dict, default {}
        Dict with keyword arguments passed to `dask.compute`.

    Other Parameters
    -----------------
    convert_func : Function
        Callback like convert_wind, convert_pv


    Returns
    -------
    resource : xr.DataArray
        Time-series of renewable generation aggregated to buses, if
        `matrix` or equivalents are provided else the total sum of
        generated energy.
    units : xr.DataArray (optional)
        The installed units per bus in MW corresponding to `layout`
        (only if `return_capacity` is True).

    """

    func_name = convert_func.__name__.replace("convert_", "")
    logger.info(f"Convert and aggregate '{func_name}'.")
    da = convert_func(cutout.data, **convert_kwds)

    no_args = all(v is None for v in [layout, shapes, matrix])

    if no_args:
        if per_unit or return_capacity:
            raise ValueError(
                "One of `matrix`, `shapes` and `layout` must be "
                "given for `per_unit` or `return_capacity`"
            )
        if capacity_factor:
            res = da.mean("time").rename("capacity factor")
            res.attrs["units"] = "p.u."
            return maybe_progressbar(res, show_progress, **dask_kwargs)
        else:
            res = da.sum("time", keep_attrs=True)
            return maybe_progressbar(res, show_progress, **dask_kwargs)

    if matrix is not None:

        if shapes is not None:
            raise ValueError(
                "Passing matrix and shapes is ambiguous. Pass " "only one of them."
            )

        if isinstance(matrix, xr.DataArray):

            coords = matrix.indexes.get(matrix.dims[1]).to_frame(index=False)
            if not np.array_equal(coords[["x", "y"]], cutout.grid[["x", "y"]]):
                raise ValueError(
                    "Matrix spatial coordinates not aligned with cutout spatial "
                    "coordinates."
                )

            if index is None:
                index = matrix

        if not matrix.ndim == 2:
            raise ValueError("Matrix not 2-dimensional.")

        matrix = csr_matrix(matrix)

    if shapes is not None:

        geoseries_like = (pd.Series, gpd.GeoDataFrame, gpd.GeoSeries)
        if isinstance(shapes, geoseries_like) and index is None:
            index = shapes.index

        matrix = cutout.indicatormatrix(shapes, shapes_crs)

    if layout is not None:

        assert isinstance(layout, xr.DataArray)
        layout = layout.reindex_like(cutout.data).stack(spatial=["y", "x"])

        if matrix is None:
            matrix = csr_matrix(layout.expand_dims("new"))
        else:
            matrix = csr_matrix(matrix) * spdiag(layout)

    # From here on, matrix is defined and ensured to be a csr matrix.
    if index is None:
        index = pd.RangeIndex(matrix.shape[0])

    results = aggregate_matrix(da, matrix=matrix, index=index)

    if per_unit or return_capacity:
        caps = matrix.sum(-1)
        capacity = xr.DataArray(np.asarray(caps).flatten(), [index])
        capacity.attrs["units"] = "MW"

    if per_unit:
        results = (results / capacity.where(capacity != 0)).fillna(0.0)
        results.attrs["units"] = "p.u."
    else:
        results.attrs["units"] = "MW"

    if return_capacity:
        return maybe_progressbar(results, show_progress, **dask_kwargs), capacity
    else:
        return maybe_progressbar(results, show_progress, **dask_kwargs)

In [9]:
with open(r'./resources/wecgenerator/Farshore_750kW.yaml') as file:
    gen = yaml.full_load(file)

In [13]:
power_matrix =pd.DataFrame.from_dict( gen['Power_Matrix'])

In [16]:
power_matrix.to_numpy().max()

750

In [18]:
def convert_wec(ds):

    #Get power matrix
    with open(r'./resources/wecgenerator/Farshore_750kW.yaml') as file:
        gen = yaml.full_load(file)
    power_matrix =pd.DataFrame.from_dict( gen['Power_Matrix'])

    #max power
    max_pow = power_matrix.to_numpy().max()
 
    ###Round up values of Hs an Tp creating new datarrays
    Hs = np.ceil(ds.wave_height*2)/2
    Tp = np.ceil(ds.wave_period*2)/2

    #Empty dataarray of results
    da = xr.DataArray.copy(Hs)
    da[:] = np.nan
    da = da.rename('Specific power generation')
    #Call datarrays values from the above dataset Dataset. need to modify function
    #
    #data arrya with results


    for x in da.x.values:
        
        for y in da.y.values:
            
            for t in da.time.values:
                
                Hs_i= Hs.sel(x=x, y=y, time=t).values 
                Tp_i = Tp.sel(x=x, y=y, time=t).values
                if np.isnan(Hs_i) or np.isnan(Tp_i):
                    power = np.nan
                else:
                    power =power_matrix.loc[Hs_i, Tp_i]
                da.loc[dict(x= x, y= y,time = t )] = power/max_pow
                print (t, x, y)
                print (Hs_i, Tp_i, power)


    

    # da = xr.apply_ufunc(
    #     _interpolate,
    #     wnd_hub,
    #     input_core_dims=[[]],
    #     output_core_dims=[[]],
    #     output_dtypes=[wnd_hub.dtype],
    #     dask="parallelized",
    # )

    da.attrs["units"] = "MWh/MWp"
    da = da.rename("specific generation")
    return da

In [19]:
test = convert_wec(cutout.data)

AttributeError: 'Dataset' object has no attribute 'wave_height'